# Answer the basic questions posed

In [ ]:
# Fix the darn connection -- driver isn't baked into the kernel. Sigh.
%pip install psycopg2-binary
from noteable_magics.datasources import bootstrap_datasources
bootstrap_datasources()

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%%sql @2b38d1248d17413982b2e98f93c4c20e
select donor,
    sum(amount) as total_donations
from donations
group by 1
order by 2 desc
limit 15

# Individual vs Corp or PAC donations by Race

In [ ]:
%%sql @2b38d1248d17413982b2e98f93c4c20e
select
    race,
    case
        when donor_is_individual_person_or_family then 'Individual'
        else 'Corp/PAC'
    end as individual_donor,
    sum(amount) as total_donations
from donations
group by 1, 2
order by 1, 3 desc

## In-state / Out-of-state By Race

In [ ]:
%%sql @2b38d1248d17413982b2e98f93c4c20e
select
    race,
    case
        when donor_state = 'CA' then 'In State'
        else 'Out of State'
    end as "Donor State",
    sum(amount) as "Amount Donated"
from
    donations
group by 1, 2
order by 1, 2

In [ ]:
%%sql @2b38d1248d17413982b2e98f93c4c20e


## In/Out of state vs Individual/Corporate donations Rollup Matrix

In [ ]:
%%sql @2b38d1248d17413982b2e98f93c4c20e
select
    case
        when donor_state = 'CA' then 'In State'
        else 'Out of State'
    end as "In/Out of State",
    
    case
        when donor_is_individual_person_or_family then 'Individual'
        else 'Corp/PAC'
    end as "Individual or PAC/Corp Donor",
    
    sum(amount)::numeric(10,3) as "Amount Donated"
    
from donations
group by rollup(1, 2)
order by "Amount Donated" desc

In [ ]:
rollup = _
rollup = rollup.fillna("Total")
rollup

In [ ]:
%%sql @2b38d1248d17413982b2e98f93c4c20e
    select

    race, 
    case
        when donor_state = 'CA' then 'In State'
        else 'Out of State'
    end as "In/Out of State",
    
    case
        when donor_is_individual_person_or_family then 'Individual'
        else 'Corp/PAC'
    end as individual_donor,
    
    sum(amount)::numeric(10,3) as "Amount Donated"
    
from donations
group by rollup (1, 2, 3)
order by "Amount Donated" desc

## Did anyone donate to more than one candidate?

In [ ]:
%%sql @2b38d1248d17413982b2e98f93c4c20e
select donor,
    count(distinct candidate) as num_candidates_donated_to,
    string_agg(distinct candidate, ', ' order by candidate) as candidates,
    count(distinct race) num_races_donated_to,
    string_agg(distinct race, ', ' order by race) as races,
    sum(amount) as total_donated
from donations
group by 1
having count(distinct candidate) > 1
order by num_candidates_donated_to desc, total_donated desc

## Any connection between donor and candidate?

In [ ]:
%%sql @2b38d1248d17413982b2e98f93c4c20e
select
    donor,
    candidate,
    sum(amount) as total_donations,
    count(*) as num_donations
from donations
where donor_last_name = candidate_last_name
group by 1, 2
order by 3 desc, 1, 2

## Did anyone from outside CA donate?


In [ ]:
%%sql @2b38d1248d17413982b2e98f93c4c20e
select donor_state as state, sum(amount) as amount_donated
from donations
where donor_state != 'CA'
group by 1 order by 2 desc

## How far in advance of the deadline were people donating?

In [ ]:
%%sql @2b38d1248d17413982b2e98f93c4c20e
select race,
    min(donation_date)::text as earliest_donation,
    max(donation_date) - min(donation_date) as days_between_first_and_last_donation
from donations
group by 1
order by 2

In [ ]:
%%sql @2b38d1248d17413982b2e98f93c4c20e
select donation_date::timestamp,
    count(*) as num_donations,
    sum(amount) as total_donated_amount,
    avg(amount) as avg_donated_amount
from donations
where donation_date is not null
group by 1
order by 1

In [ ]:
by_date = _
by_date

In [ ]:
by_date

## How many candidates were there?

In [ ]:
%%sql @2b38d1248d17413982b2e98f93c4c20e
select
    race,
    count(distinct candidate) as count_candidates,
    string_agg(distinct candidate, ', ' order by candidate) as candidates,
    sum(amount) as total_donated_in_race
from donations
group by 1
order by 2 desc

## Total Raised By Candidate/Race

In [ ]:
%%sql @2b38d1248d17413982b2e98f93c4c20e
select race,
    candidate,
    sum(amount) as total_raised,
    count(*) as num_donations
from donations
group by 1, 2
order by 3 desc

In [ ]:
by_race_candidate = _
by_race_candidate

In [ ]:
%%sql @2b38d1248d17413982b2e98f93c4c20e
select
    race,
    candidate,
    sum(amount) as "Total donated by Corp or PAC"
from donations
where not donor_is_individual_person_or_family
group by 1, 2

In [ ]:
%%sql @2b38d1248d17413982b2e98f93c4c20e
select coalesce(donor_title, 'corp or pac') as donor_title,
sum(amount) as "Donation totals by title"
from donations
where amount is not null
group by 1
order by 2 desc

In [ ]:
%%sql @2b38d1248d17413982b2e98f93c4c20e
select
    case
        when donor_title like 'self-employed%' then 'self-imployed (all)'
        else coalesce(donor_title, 'corp or pac')
    end as donor_title,
count(*) as "Donations by title"
from donations
where amount is not null
group by 1
order by 2 desc

# Out-of-state donations?

In [ ]:
%%sql @2b38d1248d17413982b2e98f93c4c20e
select
    race,
    candidate,
    sum(amount) as "Total out of state donations",
    count(*) as "Number out of state donations"
from
    donations
where donor_state != 'CA'
group by 1, 2
order by 3 desc

In [ ]:
%%sql @2b38d1248d17413982b2e98f93c4c20e
select
    race,
    candidate,
    sum(amount) as "Total out of state donations",
    count(*) as "Number out of state donations"
from
    donations
where donor_state != 'CA'
group by 1, 2
order by 3 desc

In [ ]:
%%sql @2b38d1248d17413982b2e98f93c4c20e
select * from donations where not donor_is_individual_person_or_family order by amount desc